In [1]:
import pandas as pd
import numpy as np

from utils.data_loader import load_from_tsfile_to_dataframe
from utils.regressor_tools import process_data
#import mlflow
#from tsfeatures import tsfeatures
np.set_printoptions(threshold=np.inf)
#pd.set_option('display.max_rows', None)  
#pd.set_option('display.max_columns', None) 
from utils.personal_utils import *
from compression import *
from utils.compression_algos import *

import os
import matplotlib.pyplot as plt

Test to see if the compression code still works properly
What we should get:
- Imcreasing RSME and Compression Ratio
- RSME of dropout 0 should be nearly zero. RSME for dropout 1 should be around 1.
- Compression Ratio should make sense! Super high for dropout_value close to 1. For droput_value around 0.8, 0.9 Ratio of 3 to 5.

In [3]:
test_compression()

66it [00:01, 55.58it/s] 
100%|██████████| 42/42 [00:00<00:00, 621.13it/s]


+++ AppliancesEnergy +++
$$$ dct $$$
RMSE
0.00  5.040426526181978e-16
0.04  5.9936185917228813e-05
0.08  0.00022562168391515022
0.12  0.0005065031573571895
0.16  0.0009068448209922664
0.20  0.00143585406301341
0.24  0.0021136602076584562
0.28  0.0029188745445709826
0.32  0.003894882641346664
0.36  0.0050639930089214965
0.40  0.006413852699226404
0.44  0.00799068190153901
0.48  0.009879410603003113
0.52  0.012008669145238193
0.56  0.014591228719718088
0.60  0.017593535240536886
0.64  0.02122324578210963
0.68  0.025833203600940973
0.72  0.03158233727992173
0.76  0.03929192331272111
0.80  0.050581594666246836
0.84  0.06751192962142669
0.88  0.09564984593214898
0.92  0.15016313644217363
0.96  0.2745041232561796
1.00  0.9990079365079365


Comp-Ratio
0.00  0.6557218120307587
0.04  0.6673916897355825
0.08  0.6853040608121624
0.12  0.7080226801375192
0.16  0.7338310029613841
0.20  0.7639262706516431
0.24  0.7982084878393161
0.28  0.8359467076607737
0.32  0.8784192307692308
0.36  0.927745559077

5072it [00:11, 435.65it/s]
100%|██████████| 5048/5048 [00:05<00:00, 963.89it/s] 


+++ BeijingPM25Quality +++
$$$ dct $$$
RMSE
0.00  3.1020339354103397e-16
0.04  0.0009235151044225713
0.08  0.0032627323762632037
0.12  0.006687773086253709
0.16  0.010921777779902999
0.20  0.016003280618066853
0.24  0.022095230388412445
0.28  0.028883409181422723
0.32  0.03657711287628687
0.36  0.04546176728982016
0.40  0.05521026483539755
0.44  0.06612724422456567
0.48  0.07873058795052745
0.52  0.09267947981231331
0.56  0.10885086936038
0.60  0.12691360836727011
0.64  0.14766171411244064
0.68  0.17225179747879477
0.72  0.20033423249322493
0.76  0.233808661757177
0.80  0.27560347957611797
0.84  0.3268848383114424
0.88  0.3936995696649004
0.92  0.48558478752423595
0.96  0.6157705979471225
1.00  0.8947437929212063


Comp-Ratio
0.00  0.7307878408098442
0.04  0.7498778664916694
0.08  0.774644681054064
0.12  0.8034476617418834
0.16  0.8347452009577793
0.20  0.8694375480827489
0.24  0.9090576945175328
0.28  0.9522864609411974
0.32  1.0006189152388825
0.36  1.056544229096865
0.40  1.11831469

74it [00:00, 6104.05it/s]
100%|██████████| 61/61 [00:00<00:00, 2128.79it/s]

+++ Covid3Month +++
$$$ dct $$$
RMSE
0.00  5.221348723650389e-16
0.04  5.221348723650389e-16
0.08  5.221348723650389e-16
0.12  5.221348723650389e-16
0.16  5.221348723650389e-16
0.20  5.221348723650389e-16
0.24  5.221348723650389e-16
0.28  5.221348723650389e-16
0.32  5.221348723650389e-16
0.36  5.221348723650389e-16
0.40  5.221348723650389e-16
0.44  5.221348723650389e-16
0.48  5.221348723650389e-16
0.52  5.221348723650389e-16
0.56  5.221348723650389e-16
0.60  5.221348723650389e-16
0.64  5.222373582484862e-16
0.68  0.0004956327257091606
0.72  0.003655547679771363
0.76  0.014405264925519036
0.80  0.04078909436161095
0.84  0.09609410652021405
0.88  0.1899350399892789
0.92  0.34751183915221046
0.96  0.5725429124344278
1.00  1.0


Comp-Ratio
0.00  0.6233523417780686
0.04  0.6233523417780686
0.08  0.6233523417780686
0.12  0.6233523417780686
0.16  0.6233523417780686
0.20  0.6233523417780686
0.24  0.6233523417780686
0.28  0.6233523417780686
0.32  0.6233523417780686
0.36  0.6233523417780686
0.40

0.84  1.0066425120772946
0.88  1.2725190839694656
0.92  1.8198689956331877
0.96  3.2306201550387597
1.00  88.90666666666667




$$$ dft $$$
RMSE
0.00  5.221348723650389e-16
0.04  5.221348723650389e-16
0.08  5.221348723650389e-16
0.12  5.221348723650389e-16
0.16  5.221348723650389e-16
0.20  5.221348723650389e-16
0.24  5.221348723650389e-16
0.28  5.221348723650389e-16
0.32  5.221348723650389e-16
0.36  5.221348723650389e-16
0.40  5.221348723650389e-16
0.44  5.221348723650389e-16
0.48  5.221348723650389e-16
0.52  5.221348723650389e-16
0.56  5.221348723650389e-16
0.60  5.221348723650389e-16
0.64  5.222373582484862e-16
0.68  0.0004956327257091606
0.72  0.003655547679771363
0.76  0.014405264925519036
0.80  0.04078909436161095
0.84  0.09609410652021405
0.88  0.1899350399892789
0.92  0.34751183915221046
0.96  0.5725429124344278
1.00  1.0


Comp-Ratio
0.00  0.6233523417780686
0.04  0.6233523417780686
0.08  0.6233523417780686
0.12  0.6233523417780686
0.16  0.6233523417780686
0.20  0.6233523417780

In [4]:
data_path = "/home/sim/Desktop/TS Extrinsic Regression/data/AppliancesEnergy_TEST.ts"
dataset_array = load_dataset(data_path)
dataset_id = os.path.basename(data_path).split('_')[0]

66it [00:01, 61.03it/s] 
100%|██████████| 42/42 [00:00<00:00, 659.45it/s]


In [5]:
# Check if RMSE increases with increasing dropout_ratio
for i in np.arange(0, 1.04, 0.1):
    decompressed_dataset = compress_dataset(dataset_array.copy(), dataset_id, True, "dwt", i) 
    print(i, " " ,compute_avg_rmse_of_dataset(dataset_array, decompressed_dataset))



0.0   5.040426526181978e-16
0.1   0.0003464869636634713
0.2   0.00143585406301341
0.30000000000000004   0.0033854854425775997
0.4   0.006413852699226404
0.5   0.010901034462780138
0.6000000000000001   0.017593535240536886
0.7000000000000001   0.02851264206464209
0.8   0.050581594666246836
0.9   0.11883603009165886
1.0   0.9990079365079365


In [6]:
# Check that Compression Ratio increases with increasing dropout_ratio
for i in np.arange(0, 1, 0.1):
    decompressed_dataset = compress_dataset(dataset_array.copy(), dataset_id, False, "dwt", i) 
    print(calculateCompRatio(dataset_array, decompressed_dataset))


0.6557218120307587
0.695904811237393
0.7639262706516431
0.8564384095877723
0.9828960392488775
1.16226921432788
1.4336497055980315
1.8868002985099288
2.7859338532475664
5.364644258097855


In [7]:
# Test if reshaping from (len_flat_dim, dim) to (num_dp, len_ts, dim) is working
num_dp = 2
len_ts = 5
num_dim = 4

array = np.arange(num_dp * len_ts * num_dim).reshape(num_dp, len_ts, num_dim)
print(array)

# Reshape to (len_flat_dim, dim)
array_flatdim = array.reshape(num_dp * len_ts, num_dim)
print(array_flatdim)

# Put column after column. For gzip part.
print(array_flatdim.reshape(-1, order='F'))


# Reshape back to (num_dp, len_ts, dim)
array_back = array_flatdim.reshape(-1, len_ts, num_dim)
print(array_back)

print(array_back.flatten())

[[[ 0  1  2  3]
  [ 4  5  6  7]
  [ 8  9 10 11]
  [12 13 14 15]
  [16 17 18 19]]

 [[20 21 22 23]
  [24 25 26 27]
  [28 29 30 31]
  [32 33 34 35]
  [36 37 38 39]]]
[[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]
 [12 13 14 15]
 [16 17 18 19]
 [20 21 22 23]
 [24 25 26 27]
 [28 29 30 31]
 [32 33 34 35]
 [36 37 38 39]]
[ 0  4  8 12 16 20 24 28 32 36  1  5  9 13 17 21 25 29 33 37  2  6 10 14
 18 22 26 30 34 38  3  7 11 15 19 23 27 31 35 39]
[[[ 0  1  2  3]
  [ 4  5  6  7]
  [ 8  9 10 11]
  [12 13 14 15]
  [16 17 18 19]]

 [[20 21 22 23]
  [24 25 26 27]
  [28 29 30 31]
  [32 33 34 35]
  [36 37 38 39]]]
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39]


In [8]:
# Test binary conversion and compression
num_dp = 2
len_ts = 2
num_dim = 2

array = np.arange(num_dp * len_ts * num_dim).reshape(num_dp, len_ts, num_dim)
array_flat = array.reshape(num_dp * len_ts, num_dim)

# Test if .tobytes saves metadata of the np.array -> No it flattens the array row after row (and slice after slice), then just saves the content of the array as bytes!
print(array_flat.tobytes() == np.arange(8).tobytes())


byte_nparray = np.arange(5)
print(byte_nparray.dtype)

byte_nparray = byte_nparray.tobytes()

print(bytearray)
print(len(byte_nparray))

# len(byte-string) -> return number of bytes in byte-object!


True
int64
<class 'bytearray'>
40


Test if saving with tofile keeps the array the same!

In [9]:
# Test Saving in Files

np.save('time_series.npy', train_x_p)
train_x_p[0][0][0].dtype


flattened = train_x_p.flatten()
flattened.tofile("data.bin")

train_after = np.fromfile("data.bin", dtype=train_x_p[0][0][0].dtype)
train_after.shape
if np.array_equal(flattened, train_after):
    print("The arrays have the same content.")
else:
    print("The arrays do not have the same content.")

NameError: name 'train_x_p' is not defined

In [ ]:
# Only preparing Data for Testing!

def prepare_data(data_x_p, data_y):

    # Swap the dimensions so that columns are stacked after each other. Copy since swapaxes only returns a view
    #(95, 144, 24) -> (95, 24, 144), first column gets first row etc. One Row is the ts of the belonging dimension.
    data_swapped = data_x_p.swapaxes(1, 2).copy()

    # Reshape to flattened ts. Stack the rows behind the other for each slice.
    data_x_flattend = data_swapped.reshape(data_swapped.shape[0], -1)
    

    prep_data = pd.DataFrame(data_x_flattend)
    prep_data['target'] = data_y
    prep_data.columns = prep_data.columns.astype(str) #fwiz or flaml needs string as columns!

   
    #data_x_p = data_x_p[0:2,...]

    num_datapoints = data_x_p.shape[0]
    len_timeseries = data_x_p.shape[1]
    num_dimensions = data_x_p.shape[2]
    num_features = 38

    all_features = np.ndarray((num_datapoints, num_features * num_dimensions))

    for i in range(0, num_datapoints):
        start_index = 0

        for j in range(0, num_dimensions):
            curr_ts = data_x_p[i,:,j]

            #print(curr_ts.size)

            timeseries_df = pd.DataFrame({'unique_id' : np.ones(len_timeseries),'ds': np.arange(0, len_timeseries) , 'y': curr_ts})
            
            feature_array = tsfeatures(timeseries_df, freq=1).fillna(0).values

            #print(feature_array.size)
            #print(np.isnan(feature_array).sum())

            end_index = start_index + feature_array.size
            all_features[i, start_index: end_index] = feature_array
            start_index = end_index
        

    all_features = pd.DataFrame(all_features)

    # name the features
    for i, col in enumerate(all_features.columns):
        # Generate the new column name
        new_col_name = 'f' + str(i + 1)
        # Rename the column
        all_features.rename(columns={col: new_col_name}, inplace=True)



    ts_and_features = pd.concat([prep_data, all_features], axis=1)


    all_features['target'] = data_y

    ts_and_features = pd.concat([prep_data.drop(columns=['target']), all_features], axis=1)
    
    #prep_data.columns = prep_data.columns.astype(str)

    return ts_and_features, all_features

Test preparing part function of load_and_prepare_everything

In [ ]:
# Simple test:
#-> No a perfect test, but it seems to work! I have no idea about better tests!

len_ts = 3
ts_1_v = np.arange(len_ts)
np.random.seed(42)  # For reproducibility
ts_2_v = np.random.rand(len_ts)
x = np.linspace(0, 2 * np.pi, len_ts)
ts_3_v = np.sin(x)

ts_4_v =  ts_1_v + 0.2 * ts_2_v


# Reulting Feature matrix
ts_1 = pd.DataFrame({'unique_id' : np.ones(len_ts),'ds': np.arange(0,len_ts) , 'y': ts_1_v})
ts_2 = pd.DataFrame({'unique_id' : np.ones(len_ts),'ds': np.arange(0,len_ts) , 'y': ts_2_v})
ts_3 = pd.DataFrame({'unique_id' : np.ones(len_ts),'ds': np.arange(0,len_ts) , 'y': ts_3_v})
ts_4 = pd.DataFrame({'unique_id' : np.ones(len_ts),'ds': np.arange(0,len_ts) , 'y': ts_4_v})

feature_matrix = np.vstack((tsfeatures(ts_1, freq=1).fillna(0).values, tsfeatures(ts_2, freq=1).fillna(0).values, tsfeatures(ts_3, freq=1).fillna(0).values,tsfeatures(ts_4, freq=1).fillna(0).values))
#print(feature_matrix)


layer1 = np.stack((ts_1_v, ts_2_v), axis=1)
layer2 = np.stack((ts_3_v,ts_4_v), axis=1)

input = np.stack((layer1, layer2), axis=0)


print(input)
print(feature_matrix)
tsf, f = prepare_data(input, np.arange(2))
print(tsf)
#print(f)


[[[ 0.00000000e+00  3.74540119e-01]
  [ 1.00000000e+00  9.50714306e-01]
  [ 2.00000000e+00  7.31993942e-01]]

 [[ 0.00000000e+00  7.49080238e-02]
  [ 1.22464680e-16  1.19014286e+00]
  [-2.44929360e-16  2.14639879e+00]]]
[[ 1.00000000e+00  5.00000000e-01  3.00000000e+00  4.44089210e-16
   3.33333333e-01  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  1.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   7.35200499e+02  0.00000000e+00  9.99999985e-01  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   1.00000000e+00 -0.00000000e+00  1.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 1.00000000e+00  6.66801831e-01  3.00000000e+00 -2.75921612e+00
   3.33333333e-01  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  

In [ ]:
# Create Sample TS
linear_data = linear_data = np.arange(360).reshape(3, 30, 4)  # Creates an array with values from 0 to 1199
linear_data

np.random.seed(42)  # For reproducibility
random_data = np.random.rand(3, 30, 4)
random_y = np.random.rand(3)
random_data

prepare_data(linear_data, random_y)

(     0    1    2    3    4    5    6    7    8    9  ...      f144  f145  \
 0    0    4    8   12   16   20   24   28   32   36  ...  0.521199   1.0   
 1  120  124  128  132  136  140  144  148  152  156  ...  0.521199   1.0   
 2  240  244  248  252  256  260  264  268  272  276  ...  0.521199   1.0   
 
    f146  f147      f148      f149      f150      f151     f152    target  
 0   1.0   0.9  2.931081 -0.360864  0.333636 -0.568267  0.53624  0.388170  
 1   1.0   0.9  2.931081 -0.360864  0.333636 -0.568267  0.53624  0.643288  
 2   1.0   0.9  2.931081 -0.360864  0.333636 -0.568267  0.53624  0.458253  
 
 [3 rows x 273 columns],
     f1        f2    f3   f4        f5   f6   f7   f8        f9  f10  ...  \
 0  1.0  0.985647  30.0  0.0  1.098661  0.0  0.0  0.0  1.290323  0.0  ...   
 1  1.0  0.985647  30.0  0.0  1.098661  0.0  0.0  0.0  1.290323  0.0  ...   
 2  1.0  0.985647  30.0  0.0  1.098661  0.0  0.0  0.0  1.290323  0.0  ...   
 
        f144  f145  f146  f147      f148      f14